Step 1: Install Dependencies (Colab Cell)

In [1]:
!pip install groq python-dotenv

Step 2: Setup API Key

In [2]:
import os
from getpass import getpass

os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


Step 3: Import + Initialize Client

In [3]:
from groq import Groq
import os

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

Step 4: Define the Experts (MoE Config)

In [10]:
MODEL_NAME = "llama-3.3-70b-versatile"

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """
You are a Senior Technical Support Engineer.
Be precise, logical, and code-focused.
Explain bugs clearly.
Provide corrected code snippets when possible.
Avoid unnecessary fluff.
"""
    },
    "billing": {
        "system_prompt": """
You are a Billing and Payments Specialist.
Be empathetic and professional.
Explain refund policies clearly.
Focus on subscription, payments, invoices, and charges.
Avoid technical jargon.
"""
    },
    "general": {
        "system_prompt": """
You are a helpful customer support assistant.
Answer casually but professionally.
Handle general questions and conversations.
"""
    },
    "crypto": {   # 👈 NEW TOOL EXPERT
        "tool": "get_bitcoin_price"
    }
}

Step 5: The Router (Core MoE Component)

In [11]:
def route_prompt(user_input: str) -> str:
    routing_prompt = f"""
Classify this text into one of these categories:

technical
billing
general
crypto

Return ONLY one word.
No explanation.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a strict classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    return response.choices[0].message.content.strip().lower()

Step 6: The Orchestrator

In [12]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)

    print("🔎 Routed to:", category)

    # TOOL EXPERT HANDLING
    if category == "crypto":
        return get_bitcoin_price()

    # LLM EXPERTS
    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

Step 7: Test the MoE Router

In [13]:
# Technical Test
print(process_request("My python script is throwing an IndexError on line 5."))

print("\n" + "="*60 + "\n")

# Billing Test
print(process_request("I was charged twice for my subscription this month."))

print("\n" + "="*60 + "\n")

# General Test
print(process_request("What are your working hours?"))

🔎 Routed to: technical
To assist you with the IndexError on line 5 of your Python script, I'll need more information. Please provide the following:

1. The code snippet surrounding line 5, including any relevant variable declarations or function calls.
2. The exact error message you're seeing, including the full traceback.

With this information, I can help you identify the cause of the IndexError and provide a corrected code snippet if possible.


🔎 Routed to: billing
I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you.

Please don't worry, I'm here to help resolve this issue for you. I'll do my best to assist you in getting a refund for the duplicate charge as soon as possible.

To start, could you please confirm your subscription details with me, such as your subscription plan and the date you were charged? This will help me to look into the matter further and ensure that I'm addressing the correct iss

the Tool (Mock Version)

In [14]:
import random

def get_bitcoin_price():
    # Simulated dynamic value
    mock_price = round(random.uniform(60000, 70000), 2)
    return f"The current price of Bitcoin is ${mock_price} USD."

In [15]:
print(process_request("What is the current price of Bitcoin?"))

🔎 Routed to: crypto
The current price of Bitcoin is $60119.28 USD.


In [16]:
print(process_request("My code has a syntax error."))
print(process_request("I need a refund."))

🔎 Routed to: technical
To identify and fix the syntax error, I'll need to see the code. Please provide the code snippet that's causing the issue, along with the error message you're receiving. This will help me pinpoint the problem and provide a corrected solution.
🔎 Routed to: billing
I'm so sorry to hear that you're not satisfied with our service. I'm here to help you with a refund. Can you please tell me a little bit more about your situation? What type of subscription or payment are you looking to get a refund for?

Also, I just want to let you know that our refund policy is designed to be fair and flexible. We offer refunds for unused or unwanted services, and we'll do our best to process your refund as quickly as possible.

To get started, could you please provide me with your account information or the invoice number associated with the payment you'd like to get a refund for? This will help me look into your account and determine the best course of action for your refund.


```
Architecture:

User Input
   ↓
Router LLM
   ↓
Category:
   technical
   billing
   general
   crypto   ← NEW
   ↓
If crypto → Call Tool
Else → Call LLM Expert
```